In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
import re

In [ ]:
#create a database to store students and competency data
def clean_competency_code(code):
    # Extract only alphabetic characters using regex
    match = re.search(r'[a-zA-Z]+', str(code))
    if match:
        return match.group(0)  # Return the matched alphabetic part
    else:
        return ''  # Return empty string if no alphabetic characters found

rating_df = pd.read_csv("Evaluations.txt", sep="|", skipinitialspace=True, converters={'competency_code': clean_competency_code})
rating_df.to_csv('Evaluation.csv', index=False)
grades_df = pd.read_csv('StudentCompetencies.txt', sep = "|", skipinitialspace=True, converters={'competency_code': clean_competency_code})
grades_df.to_csv('StudentCompetencies.csv', index=False)

# strip the columns so that the space is gone
rating_df.columns = rating_df.columns.str.strip()
grades_df.columns = grades_df.columns.str.strip()

#rename the id to make it the same both file
grades_df.rename(columns={"id": "student_id"}, inplace=True)

print("rating_df columns:", rating_df.columns)
print("grades_df columns:", grades_df.columns)
# Merge both files into a single CSV
student_data = pd.merge(rating_df, grades_df, on=["student_id", "competency_code"], how="inner")
student_data.to_csv("students_data.csv", index=False)
print("Student Data:")
print(student_data.head())
# # get the folder of the competencies document
# folder_path = 'doc'

# # List to store competency data
# competency_list = []

# # Read each TXT file and extract competency data
# for filename in os.listdir(folder_path):
#     if filename.endswith(".txt"):
#         with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as file:
#             lines = file.readlines()
#             # Extract details using regex or keywords
#             competency_name = lines[0].strip().replace("Details for Competency", "").strip()
#             competency_name = competency_name.split(" ")[0]

#             identity_code = re.search(r"IDENTITY CODE:\s*(.*)", " ".join(lines))
#             identity_code = identity_code.group(1) if identity_code else None

#             required = re.search(r"REQUIRED\?\s*(.*)", " ".join(lines))
#             required = required.group(1).strip().lower() == "true" if required else False

#             credits = re.search(r"CREDITS:\s*(\d+)", " ".join(lines))
#             credits = int(credits.group(1)) if credits else None

#             # Extract description dynamically
#             description_lines = []
#             recording = False  # Flag to start recording description lines

#             for line in lines:
#                 if "DESCRIPTION" in line:
#                     recording = True  # Start recording after "DESCRIPTION"
#                     continue
#                 if "RESPONSIBLE INSTRUCTOR:" in line:
#                     break  # Stop recording when reaching "RESPONSIBLE INSTRUCTOR:"
#                 if recording:
#                     description_lines.append(line)

#             description = " ".join(description_lines).strip()

#             instructor = re.search(r"RESPONSIBLE INSTRUCTOR:\s*(.*)", " ".join(lines))
#             instructor = instructor.group(1).strip() if instructor else None

#             prerequisites = re.search(r"PREREQUISITES\s*(.*)", " ".join(lines))
#             prerequisites = prerequisites.group(1).strip() if prerequisites else "None"

#             # Store extracted data
#             competency_list.append([competency_name, identity_code, required, credits, description, instructor, prerequisites])

# # Convert to DataFrame
# competency_df = pd.DataFrame(competency_list, columns=["Competency", "Identity_Code", "Required", "Credits", "Description", "Instructor", "Prerequisites"])

# # Save to CSV
# competency_df.to_csv("competencies_data.csv", index=False)
# print("Competency Data:")
# print(competency_df.head())
# Load the dataset
competency_df = pd.read_csv(
    "AllCompetenciesWithDescriptionV3.txt",
    sep="|",
    engine='python',
    skipinitialspace=True,
    on_bad_lines='skip'  # if some lines are malformed
)

# Clean column names (strip leading/trailing spaces)
competency_df.columns = [col.strip() for col in competency_df.columns]

# Drop rows where competency_code is NaN or just "---"
competency_df = competency_df[competency_df["competency_code"].notnull()]
competency_df = competency_df[~competency_df["competency_code"].str.contains("-{3,}", na=False)]

# Drop rows where title or description is missing (optional)
competency_df = competency_df[competency_df["title"].notnull()]
competency_df = competency_df[competency_df["description"].notnull()]

# Clean whitespace from all string columns
competency_df = competency_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Convert 'required' to boolean if needed
competency_df["required"] = competency_df["required"].map({"t": True, "f": False})

# Reset index
competency_df.reset_index(drop=True, inplace=True)
# Preview the data
print("Loaded competency_df:")
print(competency_df.head())

print("competency column:")
print(competency_df.columns)

rating_df columns: Index(['student_id', 'competency_code', 'overall', 'teaching', 'assess',
       'instructor'],
      dtype='object')
grades_df columns: Index(['student_id', 'competency_code', 'mastery_level'], dtype='object')
Student Data:
                              student_id   competency_code  overall  teaching  \
0  00f0dc1e-ba6a-497f-af18-d123c57a3811   AIC-304               5.0       5.0   
1  00f0dc1e-ba6a-497f-af18-d123c57a3811   HCD-301               5.0       5.0   
2  00f0dc1e-ba6a-497f-af18-d123c57a3811   HCD-501               5.0       5.0   
3  00f0dc1e-ba6a-497f-af18-d123c57a3811   MAT-201               5.0       5.0   
4  00f0dc1e-ba6a-497f-af18-d123c57a3811   MAT-207               5.0       5.0   

   assess  instructor  mastery_level  
0     5.0         5.0            4.0  
1     5.0         5.0            4.0  
2     5.0         5.0            3.0  
3     5.0         5.0            4.0  
4     5.0         5.0            3.0  
Loaded competency_df:
  competency_c

<ipython-input-25-911d6ffcfce2>:105: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  competency_df = competency_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


get rid of all softskill before put it into the database


In [ ]:
#  remove soft skills from all of data frame
competency_df = competency_df[~competency_df["competency_code"].str.startswith("SOF")]
# Convert 'competency_code' to string type before applying str.startswith
grades_df['competency_code'] = grades_df['competency_code'].astype(str)
grades_df = grades_df[~grades_df["competency_code"].str.startswith("SOF")]
# Convert 'competency_code' to string type before applying str.startswith
rating_df['competency_code'] = rating_df['competency_code'].astype(str)
rating_df = rating_df[~rating_df["competency_code"].str.startswith("SOF")]


create a database in sqlite

In [ ]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect("a1ce_recommendation.db")
cursor = conn.cursor()

#get rid of the table that is not correct
tables_to_drop = ["student", "competency_data", "Competency_prerequisites", "Competency_similarity", "Recommended_roadmap"]
# Drop tables if they exist
for table in tables_to_drop:
    try:
        cursor.execute(f"DROP TABLE IF EXISTS {table}")
        print(f"Table '{table}' dropped successfully.")
    except sqlite3.OperationalError as e:
        print(f"Error dropping table '{table}': {e}")

# Create tables based on the schema
cursor.execute("""
CREATE TABLE student (
    student_id TEXT,
    competency_code TEXT,
    Overall_rating INTEGER DEFAULT 0,
    Grade TEXT DEFAULT NULL,
    PRIMARY KEY (student_id, competency_code),
    FOREIGN KEY (competency_code) REFERENCES competency_data(competency_code)
);
""")

cursor.execute("""
CREATE TABLE competency_data (
    competency_code TEXT PRIMARY KEY,
    title TEXT DEFAULT NULL,
    description TEXT DEFAULT NULL,
    pillar_prefix TEXT DEFAULT NULL,
    domain_id TEXT DEFAULT NULL,
    domain_title TEXT DEFAULT NULL,
    area_id TEXT DEFAULT NULL,
    area_title TEXT DEFAULT NULL,
    credits INTEGER DEFAULT 0,
    required BOOLEAN DEFAULT FALSE,
    Open BOOLEAN DEFAULT FALSE
);
""")

cursor.execute("""
CREATE TABLE Competency_prerequisites (
    competency_code TEXT,
    prerequisite_code TEXT,
    PRIMARY KEY (competency_code, prerequisite_code),
    FOREIGN KEY (competency_code) REFERENCES competency_data(competency_code),
    FOREIGN KEY (prerequisite_code) REFERENCES competency_data(competency_code)
);
""")

cursor.execute("""
CREATE TABLE Competency_similarity (
    competency_code_1 TEXT,
    competency_code_2 TEXT,
    similarity_score_from_topic_modeling FLOAT DEFAULT NULL,
    similarity_score_from_content_base FLOAT DEFAULT NULL,
    PRIMARY KEY (competency_code_1, competency_code_2),
    FOREIGN KEY (competency_code_1) REFERENCES competency_data(competency_code),
    FOREIGN KEY (competency_code_2) REFERENCES competency_data(competency_code)
);
""")

cursor.execute("""
CREATE TABLE Recommended_roadmap (
    student_id TEXT,
    competency_code TEXT,
    PRIMARY KEY (student_id, competency_code),
    FOREIGN KEY (student_id) REFERENCES student(student_id),
    FOREIGN KEY (competency_code) REFERENCES competency_data(competency_code)
);
""")

# Commit and close connection
conn.commit()
conn.close()

print("SQLite database and tables created successfully.")




Table 'student' dropped successfully.
Table 'competency_data' dropped successfully.
Table 'Competency_prerequisites' dropped successfully.
Table 'Competency_similarity' dropped successfully.
Table 'Recommended_roadmap' dropped successfully.
SQLite database and tables created successfully.


store dataframes into the database

In [ ]:
import re
# Connect to the SQLite database
conn = sqlite3.connect("a1ce_recommendation.db")
cursor = conn.cursor()

# Store rating data (Evaluation.txt)
for index, row in rating_df[1:].iterrows():
    student_id = str(row["student_id"]).strip()
    student_id_match = re.findall(r'[a-zA-Z0-9]+', student_id)
    student_id = ''.join(student_id_match)
    competency_code = str(row["competency_code"]).replace(" ", "")  # Remove spaces  # Ensure it's a string
    # Extract ALL alphanumeric characters (letters + numbers)
    matches = re.findall(r'[a-zA-Z0-9-]+', competency_code)
    competency_code = ''.join(matches)  # Join multiple parts if found
    cursor.execute("""
      INSERT INTO student (student_id, competency_code, Overall_rating)
      VALUES (?, ?, ?)
      ON CONFLICT(student_id, competency_code)
      DO UPDATE SET Overall_rating = EXCLUDED.Overall_rating;
    """, (student_id, competency_code, row["overall"]))

# Store student competency data (StudentCompetencies.txt)
for index, row in grades_df[1:].iterrows():
    student_id = str(row["student_id"]).strip()
    student_id_match = re.findall(r'[a-zA-Z0-9]+', student_id)
    student_id = ''.join(student_id_match)
    competency_code = str(row["competency_code"]).rstrip().replace(" ", "")  # Remove spaces  # Ensure it's a string
    # Extract ALL alphanumeric characters (letters + numbers)
    matches = re.findall(r'[a-zA-Z0-9-]+', competency_code)
    competency_code = ''.join(matches)  # Join multiple parts if found
    cursor.execute("""
      INSERT INTO student (student_id, competency_code, Grade)
      VALUES (?, ?, ?)
      ON CONFLICT(student_id, competency_code)
      DO UPDATE SET Grade = EXCLUDED.Grade;
    """, (student_id, competency_code, row["mastery_level"]))


# Store competency details (from doc/ folder)
for index, row in competency_df.iterrows():
    cursor.execute("""
      INSERT INTO competency_data ( competency_code, title, description,
      pillar_prefix, domain_id, domain_title,
      area_id, area_title, credits, required)
      VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
      ON CONFLICT(competency_code)
      DO UPDATE SET
      required = EXCLUDED.required,
      credits = EXCLUDED.credits,
      description = EXCLUDED.description;
    """, (
      row["competency_code"],
      row["title"],
      row["description"],
      row["pillar_prefix"],
      row["domain_id"],
      row["domain_title"],
      row["area_id"],
      row["area_title"],
      int(row["credits"]) if pd.notna(row["credits"]) else 0,
      bool(row["required"]),
        ))

# Commit and close connection
conn.commit()

# Fetch and print student data
print("\n--- Student Table Data ---")
cursor.execute("SELECT * FROM student LIMIT 5")
students = cursor.fetchall()
for student in students:
    print(student)

# Fetch and print competency data
print("\n--- Competency Table Data ---")
cursor.execute("SELECT * FROM competency_data LIMIT 5")
competencies = cursor.fetchall()
for competency in competencies:
    print(competency)

# Count and print total records inserted
cursor.execute("SELECT COUNT(*) FROM student")
student_count = cursor.fetchone()[0]
cursor.execute("SELECT COUNT(*) FROM competency_data")
competency_count = cursor.fetchone()[0]

print(f"\nTotal students inserted: {student_count}")
print(f"Total competencies inserted: {competency_count}")

conn.close()

print("All data inserted successfully!")

ProgrammingError: Cannot operate on a closed database.

Check student data inside of the database

In [ ]:
# Connect to the database
conn = sqlite3.connect('a1ce_recommendation.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM student")
student_data = cursor.fetchall()

print("Student Data:")
print(student_data)
#print number of student that is in the database
print("Number of students in the database:", len(student_data))

Student Data:
[('00f0dc1eba6a497faf18d123c57a3811', 'AIC-304', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'ENI-405', 5, None), ('00f0dc1eba6a497faf18d123c57a3811', 'HCD-301', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'HCD-501', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'MAT-104', 5, None), ('00f0dc1eba6a497faf18d123c57a3811', 'MAT-201', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'MAT-207', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SCI-105', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEC-201', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEC-301', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-103', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-107', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-108', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-208', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-209', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-311', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-3

In [ ]:


# Connect to your database
conn = sqlite3.connect("a1ce_recommendation.db")
cursor = conn.cursor()

# Read the competency data into a DataFrame
query = "SELECT competency_code, description FROM competency_data"
competency_df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()


# Get a list of competency names
competency_titles = competency_df["competency_code"].tolist()

# Display the dataset
print("Dataset:")
print(competency_df)



Dataset:
    competency_code                                        description
0           AIC-101  This competency considers the definition of ar...
1           AIC-201  This competency introduces fundamental machine...
2           AIC-301  This competency provides you with an understan...
3           AIC-302  This competency introduces students to a varie...
4           AIC-303  This competency introduces the core AI concept...
..              ...                                                ...
148         URD-322  This competency is intended to give students p...
149         URD-401  The undergraduate research thesis competency r...
150         URD-402  The undergraduate research thesis competency r...
151         URD-411  The Undergraduate Capstone Project integrates ...
152         URD-412  The Undergraduate Capstone Project integrates ...

[153 rows x 2 columns]


input the prerequisite data into the database

In [ ]:
import sqlite3

# Step 1: Read and parse the TXT file
prerequisites = []
with open("PrerequisitesV3.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Skip the header line and divider line
for line in lines[2:]:
    parts = line.strip().split("|")
    if len(parts) != 2:
        continue  # Skip malformed lines

    competency_code = parts[0].strip()
    prereqs_raw = parts[1].strip().strip("{} ")

    if prereqs_raw:  # Non-empty prereq list
        prereq_list = [p.strip() for p in prereqs_raw.split(",")]
        for prereq_code in prereq_list:
            prerequisites.append((competency_code, prereq_code))

# Step 2: Insert into the database
conn = sqlite3.connect("a1ce_recommendation.db")
cursor = conn.cursor()

for competency_code, prereq_code in prerequisites:
    cursor.execute("""
        INSERT OR IGNORE INTO Competency_prerequisites (competency_code, prerequisite_code)
        VALUES (?, ?)
    """, (competency_code, prereq_code))

conn.commit()
conn.close()

print("Prerequisite data inserted from TXT file into the database.")


Prerequisite data inserted from TXT file into the database.


In [ ]:
# Preprocess and vectorize the descriptions using TF-IDF
# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the movie descriptions
tfidf_matrix = vectorizer.fit_transform(competency_df['description'])

# Calculate similarity matrix
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Convert similarity matrix to a DataFrame for better readability
similarity_df = pd.DataFrame(
    similarity_matrix,
    index=competency_df['competency_code'],
    columns=competency_df['competency_code']
)

print("\nSimilarity Matrix:")
print(similarity_df)



Similarity Matrix:
competency_code   AIC-101   AIC-201   AIC-301   AIC-302   AIC-303   AIC-304  \
competency_code                                                               
AIC-101          1.000000  0.008808  0.059811  0.082956  0.128078  0.024817   
AIC-201          0.008808  1.000000  0.123970  0.105589  0.022890  0.186609   
AIC-301          0.059811  0.123970  1.000000  0.024805  0.023622  0.018908   
AIC-302          0.082956  0.105589  0.024805  1.000000  0.038297  0.120818   
AIC-303          0.128078  0.022890  0.023622  0.038297  1.000000  0.060377   
...                   ...       ...       ...       ...       ...       ...   
URD-322          0.013389  0.013070  0.014303  0.021867  0.005608  0.030185   
URD-401          0.014203  0.013864  0.011113  0.023196  0.039280  0.003487   
URD-402          0.014073  0.013737  0.011011  0.022983  0.038920  0.003455   
URD-411          0.006413  0.016981  0.000000  0.010473  0.046676  0.000000   
URD-412          0.006413  0.016

We need to save the information on the similarity of each competency in a data base so that we don't need to run it everytime


In [ ]:
# Connect to the database
conn = sqlite3.connect('a1ce_recommendation.db')
cursor = conn.cursor()

# Prepare and insert data
insert_query = """
INSERT INTO Competency_similarity (competency_code_1, competency_code_2, similarity_score_from_content_base)
VALUES (?, ?, ?)
ON CONFLICT(competency_code_1, competency_code_2) DO UPDATE SET
similarity_score_from_content_base = excluded.similarity_score_from_content_base;
"""

# Iterate through the similarity DataFrame and insert values
for comp_1 in similarity_df.index:
    for comp_2 in similarity_df.columns:
        if comp_1 != comp_2:  # Optional: Skip self-similarity (diagonal)
            similarity_score = similarity_df.loc[comp_1, comp_2]
            cursor.execute(insert_query, (comp_1, comp_2, similarity_score))

# Commit and close the connection
conn.commit()

# Fetch and print competency similarity data
print("\n--- Competency Similarity Table Data ---")
cursor.execute("SELECT * FROM Competency_similarity")
competencies = cursor.fetchall()
for competency in competencies:
    print(competency)

conn.close()



--- Competency Similarity Table Data ---
('AIC-101', 'AIC-201', None, 0.008807742020749153)
('AIC-101', 'AIC-301', None, 0.05981080539806055)
('AIC-101', 'AIC-302', None, 0.08295644857533989)
('AIC-101', 'AIC-303', None, 0.1280777203127407)
('AIC-101', 'AIC-304', None, 0.02481659345958712)
('AIC-101', 'AIC-305', None, 0.036049118015680434)
('AIC-101', 'AIC-402', None, 0.0)
('AIC-101', 'AIC-403', None, 0.013889795532603097)
('AIC-101', 'AIC-502', None, 0.004784307513302179)
('AIC-101', 'AIC-503', None, 0.04039986763398175)
('AIC-101', 'AIC-504', None, 0.09329819511930601)
('AIC-101', 'AIC-601', None, 0.0)
('AIC-101', 'AIC-602', None, 0.021705162516635373)
('AIC-101', 'AIC-603', None, 0.08599412294641263)
('AIC-101', 'AIC-604', None, 0.028772675563953003)
('AIC-101', 'AIC-605', None, 0.043855211930753465)
('AIC-101', 'COM-101', None, 0.011034435643193764)
('AIC-101', 'COM-102', None, 0.0)
('AIC-101', 'COM-103', None, 0.0)
('AIC-101', 'COM-104', None, 0.008073371969239322)
('AIC-101', 'C

In [ ]:
# Nearest Neighbor for Recommendations
def nearest_neighbor_recommend(competency_title, tfidf_matrix, competency_titles, top_n=3):
    # Fit Nearest Neighbors
    model = NearestNeighbors(metric='cosine', algorithm='brute')
    model.fit(tfidf_matrix)

    # Get similarity scores for the given competency
    similar_competencies = similarity_df[competency_title].sort_values(ascending=False)

    # Exclude itself and get the top N
    recommended_competencies = similar_competencies.drop(competency_title).head(top_n).index.tolist()

    return recommended_competencies



After we get the neares neighbor, we will know which competency is similar to which competency. Then we can use Item base to create a recommendation road map by recommend the similar competency to previous one that they perform really good.


In [ ]:
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline
from zipfile import ZipFile

In [ ]:

# Fetch all of the student data
# Connect to the database
conn = sqlite3.connect('a1ce_recommendation.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM student")
student_data = cursor.fetchall()

print("Student Data:")
print(student_data)

# Ask user for student ID
student_id_input = input("Enter the student ID to generate recommendations for: ").strip()

# Get column names from the cursor description
columns = [desc[0] for desc in cursor.description]


student_df = pd.DataFrame(student_data, columns=columns)

# Filter the DataFrame to only include this student
student_df = student_df[student_df["student_id"] == student_id_input]

print("Student data frame")
print(student_df)


# Handle case where student ID is not found
if student_df.empty:
    print(f"No data found for student ID: {student_id_input}")
    exit()


# get all of the competency code
cursor.execute("SELECT competency_code, required, credits FROM competency_data")
competency_data = cursor.fetchall()
competency_codes = {}
for row in competency_data:
    competency_codes[row[0]] = {"required": row[1], "credits": row[2]}

conn.commit()
conn.close()

print("Total rows:", len(student_df))
print("Unique competencies:", student_df["competency_code"].nunique())


# Add the required column by mapping competency_code to required_data dictionary
student_df["required"] = student_df["competency_code"].map(lambda x: competency_codes.get(x, {}).get("required", 0)).fillna(0).astype(int)

# Convert 'Grade' column to numeric, handling non-numeric values
student_df["Grade"] = pd.to_numeric(student_df["Grade"], errors='coerce').fillna(0)

# we needs to find which competency that should be recommend to the student
# Normalize scores (to weigh them equally)
student_df["Score"] = (
    (student_df["Grade"] / 4) * 0.5 +  # Weight 50%
    (student_df["Overall_rating"] / 5) * 0.3 +  # Weight 30%
    (student_df["required"] * 0.2) # Weight 20%
)

print("Student data frame")
print(student_df[["competency_code" , "Score"]])
#we get the student score of that competency and it is more than 80% we will choose that competency as one of the base to look into others that related to them.
# Select competencies where the student has a Score >= 80%
top_competencies = [
    str(comp).strip() for comp in student_df[student_df["Score"] >= 0.8]["competency_code"].tolist()
]
print("top competencies: "+str(top_competencies))
# Find Related Competencies using Nearest Neighbor
recommended_competencies = set()  # Use a set to avoid duplicates

for competency in top_competencies:
    # Check if competency exists before recommending
    if competency in competency_titles:
        related = nearest_neighbor_recommend(competency, similarity_df, competency_titles, top_n=3)
        recommended_competencies.update(related)  # Add to set
    else:
        print(f"Warning: Competency '{competency}' not found. Skipping.")
# Check Prerequisites (Only Recommend Competencies Where Prerequisites Are Met)
completed_competencies = set(top_competencies)  # Student's completed competencies
eligible_recommendations = []

# Ensure eligible_recommendations is a DataFrame before iterating
recommend_df = pd.DataFrame(eligible_recommendations)

for competency in recommended_competencies:
    if competency in prerequisites:  # Check if it has prerequisites
        required_prereqs = prerequisites[competency]
        if not set(required_prereqs).issubset(completed_competencies):  # If prerequisites are missing
            continue
    eligible_recommendations.append(competency)


#filter out previously completed competency
recommend_df = pd.DataFrame(eligible_recommendations, columns=["Competency"])
# Exclude competencies the student has already completed
completed_competencies = set(student_df["competency_code"].tolist())
eligible_recommendations = [
    comp for comp in eligible_recommendations if comp not in completed_competencies
]

print("\nEligible Recommendations:")
print(eligible_recommendations)
#check how many recommended competency we get and check if it reach the credit limit(60 credits)

# Convert set to DataFrame
recommend_df = pd.DataFrame(eligible_recommendations, columns=["Competency"])

# Convert competency_codes to a DataFrame for merging
competency_codes_df = pd.DataFrame.from_dict(competency_codes, orient='index')
competency_codes_df.index.name = 'Competency'  # Set the index name
competency_codes_df.reset_index(inplace=True)  # Reset the index to make 'Competency' a column
# Merge with competency credit data using the DataFrame
recommend_df = recommend_df.merge(competency_codes_df[['Competency', 'credits', 'required']].drop_duplicates(), on="Competency", how="left")

# Apply Credit Limit Constraint (Max 60 Credits)
total_credits = 0
check_credit_recommendations = []
used_categories = set()  # To ensure diversity

# Sort by 'required' descending (True first), then by 'credits' ascending
recommend_df_sorted = recommend_df.sort_values(by=["required", "credits"], ascending=[False, True])

for _, row in recommend_df_sorted.iterrows():
    if total_credits + row["credits"] <= 60 and row["Competency"] not in used_categories:
        check_credit_recommendations.append(row)
        total_credits += row["credits"]
        used_categories.add(row["Competency"])  # Avoid recommending too many from the same category

# Print Final Recommendations
final_recommend_df = pd.DataFrame(check_credit_recommendations)
print("\nFinal Recommendations:")
print(final_recommend_df)



Student Data:
[('00f0dc1eba6a497faf18d123c57a3811', 'AIC-304', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'ENI-405', 5, None), ('00f0dc1eba6a497faf18d123c57a3811', 'HCD-301', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'HCD-501', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'MAT-104', 5, None), ('00f0dc1eba6a497faf18d123c57a3811', 'MAT-201', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'MAT-207', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SCI-105', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEC-201', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEC-301', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-103', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-107', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-108', 5, '4.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-208', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-209', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-311', 5, '3.0'), ('00f0dc1eba6a497faf18d123c57a3811', 'SEN-3